In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

import fannypack
from lib import dpf, panda_models, panda_datasets, panda_training, fusion_pf, fusion

print(torch.__version__, np.__version__)

1.4.0 1.18.1


In [3]:
# Experiment configuration
experiment_name = "pf_fusion_3"
dataset_args = {
    'use_proprioception': True,
    'use_haptics': True,
    'use_vision': True,
    'vision_interval': 2,
}

In [4]:
dynamics_trainset = panda_datasets.PandaDynamicsDataset(
    "data/gentle_push_10.hdf5",
    **dataset_args
)
dynamics_recurrent_trainset = panda_datasets.PandaSubsequenceDataset(
    "data/gentle_push_10.hdf5",
    subsequence_length=16,
    **dataset_args
)
measurement_trainset = panda_datasets.PandaMeasurementDataset(
    "data/gentle_push_10.hdf5",
    samples_per_pair=10,
    **dataset_args
)
e2e_trainset = panda_datasets.PandaParticleFilterDataset(
    "data/gentle_push_10.hdf5",
    subsequence_length=16,
    particle_count=30,
    particle_stddev=(.1, .1),
    **dataset_args
)

Parsed data: 2027 active, 363 inactive
Keeping: 363



Loaded 2400 points
Parsed data: 139 active, 11 inactive
Keeping (inactive): 11


In [ ]:
# Create models & training buddy

pf_image_model = panda_models.PandaParticleFilterNetwork(
    panda_models.PandaDynamicsModel(),
    panda_models.PandaMeasurementModel(missing_modalities=['gripper_force', 'gripper_pos']),
)

pf_force_model = panda_models.PandaParticleFilterNetwork(
    panda_models.PandaDynamicsModel(),
    panda_models.PandaMeasurementModel(missing_modalities=['image']),
)

weight_model = fusion.CrossModalWeights()

pf_fusion_model = fusion_pf.ParticleFusionModel(
    pf_image_model,
    pf_force_model,
    weight_model
)

buddy = fannypack.utils.Buddy(
    experiment_name,
    pf_fusion_model,
    optimizer_names=[
        "e2e_fusion",
        "e2e_image",
        "e2e_force",
        "dynamics_image",
        "dynamics_force",
        "dynamics_recurrent_image",
        "dynamics_recurrent_force",
        "measurement_image",
        "measurement_force",
    ]
)

# Dynamics pre-training

In [ ]:
# Dynamics pre-training! (non-recurrent)
models = [
    (pf_image_model, 'dynamics_image'),
    (pf_force_model, 'dynamics_force'),
]
dataloader = torch.utils.data.DataLoader(dynamics_trainset, batch_size=32, shuffle=True, num_workers=2)

for pf_model, optim_name in models:
    for i in range(5):
        print("Training epoch", i)
        panda_training.train_dynamics(buddy, pf_model, dataloader, log_interval=1, optim_name=optim_name)
        print()

buddy.save_checkpoint("phase_0_dynamics_pretrain")



In [ ]:
# Dynamics pre-training! (recurrent)
models = [
    (pf_image_model, 'dynamics_recurrent_image'),
    (pf_force_model, 'dynamics_recurrent_force'),
]
dataloader = torch.utils.data.DataLoader(dynamics_recurrent_trainset, batch_size=32, shuffle=True, num_workers=2)

for pf_model, optim_name in models:
    for i in range(3):
        print("Training epoch", i)
        panda_training.train_dynamics_recurrent(buddy, pf_model, dataloader, log_interval=1, loss_type='l2', optim_name=optim_name)

buddy.save_checkpoint("phase_1_dynamics_pretrain_recurrent")

# Measurement pre-training

In [ ]:
models = [
    (pf_image_model, 'measurement_image'),
    (pf_force_model, 'measurement_force'),
]
measurement_trainset_loader = torch.utils.data.DataLoader(
    measurement_trainset,
    batch_size=32,
    shuffle=True,
    num_workers=16)
for pf_model, optim_name in models:
    for i in range(5):
        print("Training epoch", i)
        panda_training.train_measurement(buddy, pf_model, measurement_trainset_loader, log_interval=20, optim_name=optim_name)

buddy.save_checkpoint("phase_2_measurement_pretrain")

# End-to-end training (individual)

In [ ]:
models = [
    (pf_image_model, 'e2e_image'),
    (pf_force_model, 'e2e_force'),
]
pf_model.freeze_measurement_model = False
pf_model.freeze_dynamics_model = True

for pf_model, optim_name in models:
    e2e_trainset_loader = torch.utils.data.DataLoader(e2e_trainset, batch_size=32, shuffle=True, num_workers=2)
    for i in range(5):
        print("Training epoch", i)
        panda_training.train_e2e(buddy, pf_model, e2e_trainset_loader, loss_type="mse", optim_name=optim_name)

# End-to-end training (fusion)

In [ ]:
optim_name = "e2e_fusion"
pf_model = pf_fusion_model
e2e_trainset_loader = torch.utils.data.DataLoader(e2e_trainset, batch_size=32, shuffle=True, num_workers=2)
for i in range(5):
    print("Training epoch", i)
    panda_training.train_e2e(buddy, pf_model, e2e_trainset_loader, loss_type="mse", optim_name=optim_name)